In [61]:
import pandas as pd
import numpy as np 
import csv
import matplotlib.pyplot as plot

In [62]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [63]:
covid_data= pd.read_csv('/content/drive/My Drive/Colab Notebooks/COVID_county_data.csv')
acs17_data= pd.read_csv('/content/drive/My Drive/Colab Notebooks/acs2017_census_tract_data.csv')
print(covid_data)
print(acs17_data)

               date      county       state     fips  cases  deaths
0        2020-01-21   Snohomish  Washington  53061.0      1     0.0
1        2020-01-22   Snohomish  Washington  53061.0      1     0.0
2        2020-01-23   Snohomish  Washington  53061.0      1     0.0
3        2020-01-24        Cook    Illinois  17031.0      1     0.0
4        2020-01-24   Snohomish  Washington  53061.0      1     0.0
...             ...         ...         ...      ...    ...     ...
1050253  2021-02-20  Sweetwater     Wyoming  56037.0   3645    34.0
1050254  2021-02-20       Teton     Wyoming  56039.0   3318     9.0
1050255  2021-02-20       Uinta     Wyoming  56041.0   2024    12.0
1050256  2021-02-20    Washakie     Wyoming  56043.0    876    26.0
1050257  2021-02-20      Weston     Wyoming  56045.0    619     5.0

[1050258 rows x 6 columns]
           TractId        State  ... FamilyWork  Unemployment
0       1001020100      Alabama  ...        0.0           4.6
1       1001020200      Alabama 

In [67]:
# Aggregrate Census Data to County

counties=["Loudoun", "Washington", "Harlan", "Malheur"]
states=["Virginia", "Oregon", "Kentucky", "Oregon"]

allrecords=[]
each=[]
for i in range(len(counties)):
  check_county=acs17_data[acs17_data['County']==counties[i]+" County"]
  county=check_county[check_county['State']==states[i]]
  totalpopulation=county['TotalPop'].sum(axis=0)
  poverty=county['Poverty'].mean(axis=0)
  percaptia=county['IncomePerCap'].mean(axis=0)

  each.append(counties[i])
  each.append(states[i])
  each.append(totalpopulation)
  each.append(poverty)
  each.append(percaptia)
  allrecords.append(each)
  each=[]

aggregated_df=pd.DataFrame.from_records(allrecords)
aggregated_df.round({"3":3,"4":3})
aggregated_df.columns = ['County','State','Population','Poverty','PerCapitaIncome']
aggregated_df.head()

,County,State,Population,Poverty,PerCapitaIncome
0,Loudoun,Virginia,374558,3.884375,50391.015625
1,Washington,Oregon,572071,10.446154,34970.817308
2,Harlan,Kentucky,27548,33.318182,16010.363636
3,Malheur,Oregon,30421,24.414286,17966.428571


In [68]:
# Simplify the Covid data

allrecords=[]
each=[]
for i in range(len(counties)):
  check_county1=covid_data[covid_data['county']==counties[i]]
  c_county=check_county1[check_county1['state']==states[i]]
  totalcases=c_county['cases'].sum(axis=0)
  dec=c_county[c_county['date'].between("2020-12-01","2020-12-31")]
  dec2020=dec['cases'].sum(axis=0)
  deaths=c_county['deaths'].sum(axis=0)
  dec2020_d=dec['deaths'].sum(axis=0)

  each.append(counties[i])
  each.append(states[i])
  each.append(totalcases)
  each.append(dec2020)
  each.append(deaths)
  each.append(dec2020_d)

  allrecords.append(each)
  each=[]

covid_df=pd.DataFrame.from_records(allrecords)
covid_df.columns = ['County','State', 'TotalCases','Dec2020Cases','TotalDeaths','Dec2020Deaths']
covid_df.head()


,County,State,TotalCases,Dec2020Cases,TotalDeaths,Dec2020Deaths
0,Loudoun,Virginia,2496450,376223,35820.0,4729.0
1,Washington,Oregon,2157339,424620,22455.0,3860.0
2,Harlan,Kentucky,205984,38959,3994.0,506.0
3,Malheur,Oregon,453634,82916,7770.0,1465.0


In [69]:
# Integrate Covid Data with ACS17 Data 

integrate_df = pd.merge(aggregated_df,covid_df,on='County',how='inner')
del integrate_df['State_y']
integrate_df.rename(columns = {'State_x':'State'}, inplace = True) 
oregon_r = integrate_df[integrate_df['State']=='Oregon']
oregon_r['TotalCases'] = (oregon_r['TotalCases']*100000)/oregon_r['Population']
oregon_r['TotalDeaths'] = (oregon_r['TotalDeaths']*100000)/oregon_r['Population']
oregon_r.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,County,State,Population,Poverty,PerCapitaIncome,TotalCases,Dec2020Cases,TotalDeaths,Dec2020Deaths
1,Washington,Oregon,572071,10.446154,34970.817308,3.771104e+05,424620,3925.212080,3860.0
3,Malheur,Oregon,30421,24.414286,17966.428571,1.491187e+06,82916,25541.566681,1465.0


In [82]:
# Analysis of all the counties in the state of oregon.
# Here just took the oregon state all counties as a dataframe
allrecords=[]
each=[]
state=covid_data[covid_data['state']=='Oregon']
d_allcounties=state['county'].unique()
allcounties=list(d_allcounties)

for i in range(len(allcounties)):
  each.append('Oregon')
  acs_county=acs17_data[acs17_data['County']==allcounties[i]+" County"]
  covid_county=covid_data[covid_data['county']==allcounties[i]]
  each.append(allcounties[i])

  totalcases=covid_county['cases'].sum(axis=0)
  dec=covid_county[covid_county['date'].between("2020-12-01","2020-12-31")]
  dec2020=dec['cases'].sum(axis=0)
  deaths=covid_county['deaths'].sum(axis=0)
  dec2020_d=dec['deaths'].sum(axis=0)

  totalpopulation=acs_county['TotalPop'].sum(axis=0)
  poverty=acs_county['Poverty'].mean(axis=0)
  percaptia=acs_county['IncomePerCap'].mean(axis=0)
  
  totalcases=(totalcases*100000)/totalpopulation
  each.append(totalcases)
  each.append(dec2020)
  totaldeath=(deaths*100000)/totalpopulation
  each.append(totaldeath)
  each.append(dec2020_d)
  each.append(totalpopulation)
  each.append(poverty)
  each.append(percaptia)

  allrecords.append(each)
  each=[]


oregon_df=pd.DataFrame.from_records(allrecords)
oregon_df=oregon_df.dropna(axis=0)
oregon_df.columns = ['State','County','TotalCases','Dec2020Cases','TotalDeaths','Dec2020Deaths','Population','Poverty','PerCapitaIncome']
oregon_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in double_scalars


,State,County,TotalCases,Dec2020Cases,TotalDeaths,Dec2020Deaths,Population,Poverty,PerCapitaIncome
0,Oregon,Washington,793187.636812,3990534,12714.035387,53730.0,2564646,13.422243,30729.954380
1,Oregon,Jackson,741655.778100,2510114,11596.367446,33952.0,1722755,18.493882,26107.727059
2,Oregon,Klamath,339689.175679,45118,4327.607622,373.0,66018,18.930000,23712.400000
3,Oregon,Douglas,938039.018649,2741060,10664.338576,27887.0,1440952,13.097015,33581.874627
4,Oregon,Marion,953441.471648,3456922,23045.232016,66247.0,2018526,19.951429,24578.778022


In [83]:
#Correlation Coefficient for Oregon dataframe

rvalues=[]
R1 = oregon_df['TotalCases'].corr(oregon_df['Poverty'])
R2 = oregon_df['TotalDeaths'].corr(oregon_df['Poverty'])
R3 = oregon_df['TotalCases'].corr(oregon_df['PerCapitaIncome'])
R4 = oregon_df['TotalDeaths'].corr(oregon_df['PerCapitaIncome'])
R5 = oregon_df['Dec2020Cases'].corr(oregon_df['Poverty'])
R6 = oregon_df['Dec2020Deaths'].corr(oregon_df['Poverty'])
R7 = oregon_df['Dec2020Cases'].corr(oregon_df['PerCapitaIncome'])
R8 = oregon_df['Dec2020Deaths'].corr(oregon_df['PerCapitaIncome'])
rvalues.append(R1)
rvalues.append(R2)
rvalues.append(R3)
rvalues.append(R4)
rvalues.append(R5)
rvalues.append(R6)
rvalues.append(R7)
rvalues.append(R8)
print(rvalues)

check_rval=[]
for i in rvalues:
  if(i>0.5 or i<-0.5):
    check_rval.append(i)
print(check_rval)

[0.297471134394528, 0.20946190596896594, -0.18950461546988256, -0.07405908604882054, 0.057059538757632346, 0.05571798839796383, 0.2851092192782961, 0.2664467664558328]
[]


In [90]:
# Analysis of all the counties in the USA
# Here we just took the all counties as a dataframe
allrecords=[]
each=[]
d_allcounties=covid_data['county'].unique()
allcounties=list(d_allcounties)

for i in range(len(allcounties)):
  acs_county=acs17_data[acs17_data['County']==allcounties[i]+" County"]
  covid_county=covid_data[covid_data['county']==allcounties[i]]
  each.append(allcounties[i])

  totalcases=covid_county['cases'].sum(axis=0)
  dec=covid_county[covid_county['date'].between("2020-12-01","2020-12-31")]
  dec2020=dec['cases'].sum(axis=0)
  deaths=covid_county['deaths'].sum(axis=0)
  dec2020_d=dec['deaths'].sum(axis=0)

  totalpopulation=acs_county['TotalPop'].sum(axis=0)
  poverty=acs_county['Poverty'].mean(axis=0)
  percaptia=acs_county['IncomePerCap'].mean(axis=0)
  
  totalcases=(totalcases*100000)/totalpopulation
  each.append(totalcases)
  each.append(dec2020)
  totaldeath=(deaths*100000)/totalpopulation
  each.append(totaldeath)
  each.append(dec2020_d)
  each.append(totalpopulation)
  each.append(poverty)
  each.append(percaptia)

  allrecords.append(each)
  each=[]


allcounties_df=pd.DataFrame.from_records(allrecords)
allcounties_df.columns = ['County','TotalCases','Dec2020Cases','TotalDeaths','Dec2020Deaths','Population','Poverty','PerCapitaIncome']
allcounties_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


,County,TotalCases,Dec2020Cases,TotalDeaths,Dec2020Deaths,Population,Poverty,PerCapitaIncome
0,Snohomish,4.587665e+05,614421,10338.591327,10640.0,771904,8.934899,36148.134228
1,Cook,1.180935e+06,11111631,33112.995036,233626.0,5261001,17.988948,33006.301287
2,Orange,8.418201e+05,6659614,13061.201671,94500.0,5133716,13.075722,35621.433298
3,Maricopa,1.267895e+06,8277933,23323.589623,140051.0,4155501,16.130955,30761.265642
4,Los Angeles,1.122602e+06,17784563,20998.113742,268598.0,10105722,17.323803,31389.413867


In [91]:
# Correlation Coefficient for all counties in USA.

rvalues=[]
R1 = allcounties_df['TotalCases'].corr(allcounties_df['Poverty'])
R2 = allcounties_df['TotalDeaths'].corr(allcounties_df['Poverty'])
R3 = allcounties_df['TotalCases'].corr(allcounties_df['PerCapitaIncome'])
R4 = allcounties_df['TotalDeaths'].corr(allcounties_df['PerCapitaIncome'])
R5 = allcounties_df['Dec2020Cases'].corr(allcounties_df['Poverty'])
R6 = allcounties_df['Dec2020Deaths'].corr(allcounties_df['Poverty'])
R7 = allcounties_df['Dec2020Cases'].corr(allcounties_df['PerCapitaIncome'])
R8 = allcounties_df['Dec2020Deaths'].corr(allcounties_df['PerCapitaIncome'])

rvalues.append(R1)
rvalues.append(R2)
rvalues.append(R3)
rvalues.append(R4)
rvalues.append(R5)
rvalues.append(R6)
rvalues.append(R7)
rvalues.append(R8)
print(rvalues)
check_rval=[]
for i in rvalues:
  if(i>0.5 or i<-0.5):
    check_rval.append(i)
print(check_rval)

[0.16690037795100285, 0.199728642737955, -0.1870637430384339, -0.1413631086967846, -0.027528196894307654, -0.02187824035470475, 0.2021547054177172, 0.21840972654694812]
[]
